In [1]:
import flash
from flash.core.data.utils import download_data
from flash.image import ObjectDetectionData, ObjectDetector
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from icevision.core import ClassMap
from flash.core.data.io.input import DataKeys
import pandas as pd
from pathlib import PosixPath
from PIL import Image
import os
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import numpy as np

In [2]:
train_images_root = '/home/konrad.czarnota/aicrowd/mask_detection/train'
train_csv_path = '/home/konrad.czarnota/aicrowd/mask_detection/train.csv'
val_images_root = '/home/konrad.czarnota/aicrowd/mask_detection/val'
val_csv_path = '/home/konrad.czarnota/aicrowd/mask_detection/val.csv'
test_images_root = '/home/konrad.czarnota/aicrowd/mask_detection/test'

In [3]:
class MaskDataset:
    
    def __init__(
        self,
        images_root,
        csv_file_path,
    ) -> None:
        self.images_root = images_root
        self.csv_file_path = csv_file_path
        
        self.image_paths = []
        self.bboxes = []
        self.labels = []
        df = pd.read_csv(self.csv_file_path)
        for i, v in df.iterrows():
            self.image_paths.append(os.path.join(self.images_root, v['ImageID'] + '.jpg'))
            bbox = [int(i) for i in v['bbox'][1:-1].replace(',', '').split(' ')]
            bbox = [{'xmin': bbox[0], 'ymin': bbox[1], 'width': bbox[2] - bbox[0], 'height': bbox[3] - bbox[1]}]
            self.bboxes.append(bbox)
            self.labels.append([v['masktype']])

In [4]:
train_dataset = MaskDataset(train_images_root, train_csv_path)
val_dataset = MaskDataset(val_images_root, val_csv_path)

In [5]:
datamodule = ObjectDetectionData.from_files(
    train_files=train_dataset.image_paths,
    train_targets=train_dataset.labels,
    train_bboxes=train_dataset.bboxes,
    val_files=val_dataset.image_paths,
    val_targets=val_dataset.labels,
    val_bboxes=val_dataset.bboxes,
    transform_kwargs=dict(image_size=(512, 512)),
    batch_size=16,
)

In [7]:
ObjectDetector.available_optimizers()

['adadelta',
 'adagrad',
 'adam',
 'adamax',
 'adamw',
 'asgd',
 'lbfgs',
 'nadam',
 'radam',
 'rmsprop',
 'rprop',
 'sgd',
 'sparseadam']

In [8]:
ObjectDetector.available_lr_schedulers()

['chainedscheduler',
 'constantlr',
 'cosineannealinglr',
 'cosineannealingwarmrestarts',
 'cycliclr',
 'exponentiallr',
 'lambdalr',
 'linearlr',
 'multiplicativelr',
 'multisteplr',
 'onecyclelr',
 'reducelronplateau',
 'sequentiallr',
 'steplr']

In [9]:
ObjectDetector.available_backbones()

{'efficientdet': ['d0',
  'd1',
  'd2',
  'd3',
  'd4',
  'd5',
  'd6',
  'd7',
  'd7x',
  'tf_d0',
  'tf_d0_ap',
  'tf_d1',
  'tf_d1_ap',
  'tf_d2',
  'tf_d2_ap',
  'tf_d3',
  'tf_d3_ap',
  'tf_d4',
  'tf_d4_ap',
  'tf_d5',
  'tf_d5_ap',
  'tf_d6',
  'tf_d7',
  'tf_d7x',
  'tf_lite0',
  'tf_lite1',
  'tf_lite2',
  'tf_lite3'],
 'faster_rcnn': ['mobilenet',
  'resnet101',
  'resnet101_fpn',
  'resnet152',
  'resnet152_fpn',
  'resnet18',
  'resnet18_fpn',
  'resnet34',
  'resnet34_fpn',
  'resnet50',
  'resnet50_fpn',
  'resnext101_32x8d',
  'resnext101_32x8d_fpn',
  'resnext50_32x4d_fpn',
  'wide_resnet101_2_fpn',
  'wide_resnet50_2_fpn'],
 'retinanet': ['resnet101_fpn',
  'resnet152_fpn',
  'resnet18_fpn',
  'resnet34_fpn',
  'resnet50_fpn',
  'resnext101_32x8d_fpn',
  'resnext50_32x4d_fpn',
  'wide_resnet101_2_fpn',
  'wide_resnet50_2_fpn'],
 'yolov5': ['extra_large',
  'extra_large_p6',
  'large',
  'large_p6',
  'medium',
  'medium_p6',
  'small',
  'small_p6']}

In [10]:
ObjectDetector.available_heads()

['efficientdet', 'faster_rcnn', 'retinanet', 'yolov5']

In [ ]:
model = ObjectDetector(head="efficientdet", backbone="d0", num_classes=datamodule.num_classes, image_size=512)

trainer = flash.Trainer(max_epochs=20, gpus=[2])
trainer.finetune(model, datamodule=datamodule, strategy=("freeze_unfreeze", 10))

In [ ]:
test_images_paths = [os.path.join(test_images_root, p) for p in os.listdir(test_images_root)]
test_datamodule = ObjectDetectionData.from_files(
    predict_files=test_images_paths,
    transform_kwargs={"image_size": 512},
    batch_size=8,
)
predictions = trainer.predict(model, datamodule=test_datamodule)

In [8]:
label_to_name = {i: n for i, n in enumerate(datamodule.labels)}

In [21]:
import cv2
import matplotlib.pyplot as plt

def vis_pred(pred):
    img_path = pred[DataKeys.INPUT][DataKeys.INPUT]
    if len(pred[DataKeys.PREDS]['bboxes']) == 0:
        bbox = {'xmin':0, 'ymin':0, 'width':1, 'height':1}
        label = 0
    else:
        bbox = pred[DataKeys.PREDS]['bboxes'][0]
        label = pred[DataKeys.PREDS]['labels'][0]
    img = cv2.imread(img_path)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    rgb = cv2.rectangle(rgb, (int(bbox['xmin']), int(bbox['ymin'])), (int(bbox['xmin'] + bbox['width']), int(bbox['ymin'] + bbox['height'])), (0, 0, 255), 2)
    rgb = cv2.putText(rgb, label_to_name[label], (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    plt.figure()
    plt.imshow(rgb)

In [ ]:
for i in range(2):
    for j in range(8):
        vis_pred(predictions[i][j])

In [9]:
def generate_submission(preds):
    frame = pd.DataFrame(columns=["ImageID", "bbox", "masktype"])
    for batch in preds:
        for pred in batch:
            img_path = pred[DataKeys.INPUT][DataKeys.INPUT]
            if len(pred[DataKeys.PREDS]['bboxes']) == 0:
                bbox = {'xmin':0, 'ymin':0, 'width':1, 'height':1}
                label = 0
            else:
                bbox = pred[DataKeys.PREDS]['bboxes'][0]
                label = pred[DataKeys.PREDS]['labels'][0]
            bbox = f"[{int(bbox['xmin'])}, {int(bbox['ymin'])}, {int(bbox['xmin'] + bbox['width'])}, {int(bbox['ymin'] + bbox['height'])}]"
            if label == 0:
                mask = 'surgical'
            else:
                mask = label_to_name[label]
            frame.loc[len(frame), :] = {"ImageID": img_path.split('/')[-1].split('.')[0], "bbox": bbox, "masktype": mask}
    return frame

In [10]:
test_df = generate_submission(predictions)

In [11]:
#!pip install aicrowd-cli
%load_ext aicrowd.magic

In [ ]:
%aicrowd login

In [13]:
!rm -rf assets
!mkdir assets
test_df.to_csv(os.path.join("assets", "submission.csv"), index=False)

In [ ]:
%aicrowd notebook submit -c mask-prediction -a assets --no-verify